In [1]:
from os import listdir
import xarray
import rasterio
import geopandas as gp
import numpy as np

In [43]:
ratio.lat.size

195

In [48]:
new_lat.size / 4

195.0

In [45]:
new_lat = np.linspace(ratio.lat[0], ratio.lat[-1], ratio.lat.size*4)

In [7]:
DATA_DIR = "../troubled_waters_data/netcdf/"
DATA_OUTPUT_DIR = "../troubled_waters_data/output_netcdf/"
metrics = ['frac_extreme', 'max_threeday_precip', 'nov_mar_percent',
            'rainfall_ratio', 'num_ros_events', 'norm_rain_on_snow',
            'SWE_total', 'et']

hist_suffix = '_now'
for rcp in ["RCP85", "RCP45"]:
    for met_index, metric in enumerate(metrics):
        metric_models = xarray.open_dataset(f"{DATA_DIR}{metric}_{rcp}.nc")
        metric_hist_models = xarray.open_dataset(F"{DATA_DIR}{metric}_{rcp}{hist_suffix}.nc")
        
        if metric == "SWE_total":
            metric_hist_models = metric_hist_models.where(metric_hist_models > 1)
        ratio = metric_models / metric_hist_models
        
        new_lat = np.linspace(ratio.lat[0], ratio.lat[-1], ratio.lat.size*8)
        new_lon = np.linspace(ratio.lon[0], ratio.lon[-1], ratio.lon.size*8)
        
        ratio.to_netcdf(f"{DATA_OUTPUT_DIR}{metric}_{rcp}.nc")

In [12]:
new_lat = np.linspace(ratio.lat[0], ratio.lat[-1], ratio.lat.size*3)
new_lon = np.linspace(ratio.lon[0], ratio.lon[-1], ratio.lon.size*3)

new = ratio.interp(lat=new_lat, lon=new_lon)

In [13]:
new.to_netcdf("test.nc")

In [48]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import fiona
import geopandas
import rioxarray
from shapely.geometry import mapping, Polygon

SHPFILE_DIR = "../troubled_waters_data/shapefiles/simplified/"

shapefile_path = SHPFILE_DIR + "CA_Bulletin_118_Groundwater_Basins_simplified.shp"
shapefile = geopandas.read_file(shapefile_path)

In [8]:
metric_data = {}
for filename in listdir(DATA_OUTPUT_DIR):
    if "RCP" in filename:
        metric_data[filename] = xarray.open_dataset(DATA_OUTPUT_DIR + filename)

In [103]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import fiona
import geopandas
import rioxarray
from shapely.geometry import mapping, Polygon

metric_name = "et_RCP45.nc"

SHPFILE_DIR = "../troubled_waters_data/shapefiles/original/"
DATA_OUTPUT_DIR = "../troubled_waters_data/output_netcdf/"

def preprocess_metric(metric_ds):
    # Interpolate to a higher resolution
    new_lat = np.linspace(metric_ds.lat[0], metric_ds.lat[-1], metric_ds.lat.size*8)
    new_lon = np.linspace(metric_ds.lon[0], metric_ds.lon[-1], metric_ds.lon.size*8)
    metric_ds = metric_ds.interp(lat=new_lat, lon=new_lon)
    # Adjust coordinates to match the shapefile coordinates
    metric_ds = metric_ds.assign_coords(lon=(((metric_ds.lon + 180) % 360) - 180)).sortby('lon')
    # Set spatial dimensions for data
    metric_ds.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
    # Specify CRS projection to match shapefile data
    metric_ds.rio.write_crs("epsg:4326", inplace=True)
    return metric_ds


def maskToGroundwaterBasins(input_path):
    metric_ds = preprocess_metric(xarray.open_dataset(input_path))
    
    # Read ing shapefile
    shapefile_path = SHPFILE_DIR + "CA_Bulletin_118_Groundwater_Basins.shp"
    shapefile = geopandas.read_file(shapefile_path)

    # Create dictionary for storing information on each region
    reg_ids = []
    reg_names = []
    reg_means = []
    reg_stds = []
    reg_mins = []
    reg_maxs = []

    for index, ID in enumerate(shapefile["OBJECTID"]):
        # Get polygon geometry for this region
        polys = [shapefile["geometry"][index]]
        # Format geometry set
        geom = { 'OBJECTID': index, 'geometry':polys}
        # Build data frame with same CRS
        geom_dataframe = geopandas.GeoDataFrame(geom, crs=shapefile.crs)
        # Mask data to region, including all regions it touches, not just encapsulates
        mask = metric_ds.rio.clip(geom_dataframe.geometry.apply(mapping), shapefile.crs, drop=True, all_touched=True)
        means = []
        stds = []
        mins = []
        maxs = []
        for model in list(mask.keys()):
            means.append(mask[model].mean().values)
            stds.append(mask[model].std().values)
            mins.append(mask[model].min().values)
            maxs.append(mask[model].max().values)
        
        reg_ids.append(ID)
        reg_names.append(shapefile["Basin_Su_1"][index])
        reg_means.append(means)
        reg_stds.append(stds)
        reg_mins.append(mins)
        reg_maxs.append(maxs)
    data_dict = {
        "basin_name": (["basin_id"], reg_names),
        "model_mean": (["basin_id", "model"], reg_means),
        "model_std": (["basin_id", "model"], reg_stds),
        "model_min": (["basin_id", "model"], reg_mins),
        "model_max": (["basin_id", "model"], reg_maxs)
    }
    coord_dict = {
        "basin_id": ("basin_id", reg_ids),
        "model": ("model", list(mask.keys()))
    }
        
    return xarray.Dataset(data_vars=data_dict, coords=coord_dict)


def maskToCounties(input_path):
    metric_ds = preprocess_metric(xarray.open_dataset(input_path))
    
    # Read ing shapefile
    shapefile_path = SHPFILE_DIR + "CA_Counties_TIGER2016.shp"
    shapefile = geopandas.read_file(shapefile_path)

    # Create dictionary for storing information on each region
    reg_ids = []
    reg_names = []
    reg_means = []
    reg_stds = []
    reg_mins = []
    reg_maxs = []

    for index, ID in enumerate(shapefile["COUNTYFP"]):
        # Get polygon geometry for this region
        polys = [shapefile["geometry"][index]]
        # Format geometry set
        geom = { 'OBJECTID': index, 'geometry':polys}
        # Build data frame with same CRS
        geom_dataframe = geopandas.GeoDataFrame(geom, crs=shapefile.crs)
        # Mask data to region, including all regions it touches, not just encapsulates
        mask = metric_ds.rio.clip(geom_dataframe.geometry.apply(mapping), shapefile.crs, drop=True, all_touched=True)
        means = []
        stds = []
        mins = []
        maxs = []
        for model in list(mask.keys()):
            means.append(mask[model].mean().values)
            stds.append(mask[model].std().values)
            mins.append(mask[model].min().values)
            maxs.append(mask[model].max().values)
        
        reg_ids.append(ID)
        reg_names.append(shapefile["NAME"][index])
        reg_means.append(means)
        reg_stds.append(stds)
        reg_mins.append(mins)
        reg_maxs.append(maxs)
    data_dict = {
        "county_name": (["county_fp"], reg_names),
        "model_mean": (["county_fp", "model"], reg_means),
        "model_std": (["county_fp", "model"], reg_stds),
        "model_min": (["county_fp", "model"], reg_mins),
        "model_max": (["county_fp", "model"], reg_maxs)
    }
    coord_dict = {
        "county_fp": ("county_fp", reg_ids),
        "model": ("model", list(mask.keys()))
    }
    return xarray.Dataset(data_vars=data_dict, coords=coord_dict)


def maskToPlaces(input_path):
    metric_ds = preprocess_metric(xarray.open_dataset(input_path))
    
    # Read ing shapefile
    shapefile_path = SHPFILE_DIR + "CA_Places_TIGER2016.shp"
    shapefile = geopandas.read_file(shapefile_path)

    # Create dictionary for storing information on each region
    reg_ids = []
    reg_names = []
    reg_means = []
    reg_stds = []
    reg_mins = []
    reg_maxs = []

    for index, ID in enumerate(shapefile["PLACEFP"]):
        # Get polygon geometry for this region
        polys = [shapefile["geometry"][index]]
        # Format geometry set
        geom = { 'OBJECTID': index, 'geometry':polys}
        # Build data frame with same CRS
        geom_dataframe = geopandas.GeoDataFrame(geom, crs=shapefile.crs)
        # Mask data to region, including all regions it touches, not just encapsulates
        mask = metric_ds.rio.clip(geom_dataframe.geometry.apply(mapping), shapefile.crs, drop=True, all_touched=True)
        means = []
        stds = []
        mins = []
        maxs = []
        for model in list(mask.keys()):
            means.append(mask[model].mean().values)
            stds.append(mask[model].std().values)
            mins.append(mask[model].min().values)
            maxs.append(mask[model].max().values)
        
        reg_ids.append(ID)
        reg_names.append(shapefile["NAME"][index])
        reg_means.append(means)
        reg_stds.append(stds)
        reg_mins.append(mins)
        reg_maxs.append(maxs)
    data_dict = {
        "place_name": (["place_fp"], reg_names),
        "model_mean": (["place_fp", "model"], reg_means),
        "model_std": (["place_fp", "model"], reg_stds),
        "model_min": (["place_fp", "model"], reg_mins),
        "model_max": (["place_fp", "model"], reg_maxs)
    }
    coord_dict = {
        "place_fp": ("place_fp", reg_ids),
        "model": ("model", list(mask.keys()))
    }
    return xarray.Dataset(data_vars=data_dict, coords=coord_dict)


def maskToWBD(input_path):
    metric_ds = preprocess_metric(xarray.open_dataset(input_path))
    
    # Read ing shapefile
    shapefile_path = SHPFILE_DIR + "WBD_USGS_HUC10_CA.shp"
    shapefile = geopandas.read_file(shapefile_path)

    # Create dictionary for storing information on each region
    reg_ids = []
    reg_names = []
    reg_means = []
    reg_stds = []
    reg_mins = []
    reg_maxs = []

    for index, ID in enumerate(shapefile["OBJECTID"]):
        # Get polygon geometry for this region
        polys = [shapefile["geometry"][index]]
        # Format geometry set
        geom = { 'OBJECTID': index, 'geometry':polys}
        # Build data frame with same CRS
        geom_dataframe = geopandas.GeoDataFrame(geom, crs=shapefile.crs)
        # Mask data to region, including all regions it touches, not just encapsulates
        mask = metric_ds.rio.clip(geom_dataframe.geometry.apply(mapping), shapefile.crs, drop=True, all_touched=True)
        means = []
        stds = []
        mins = []
        maxs = []
        for model in list(mask.keys()):
            means.append(mask[model].mean().values)
            stds.append(mask[model].std().values)
            mins.append(mask[model].min().values)
            maxs.append(mask[model].max().values)
        
        reg_ids.append(ID)
        reg_names.append(shapefile["Name"][index])
        reg_means.append(means)
        reg_stds.append(stds)
        reg_mins.append(mins)
        reg_maxs.append(maxs)
    data_dict = {
        "wbd_name": (["wbd_id"], reg_names),
        "model_mean": (["wbd_id", "model"], reg_means),
        "model_std": (["wbd_id", "model"], reg_stds),
        "model_min": (["wbd_id", "model"], reg_mins),
        "model_max": (["wbd_id", "model"], reg_maxs)
    }
    coord_dict = {
        "wbd_id": ("wbd_id", reg_ids),
        "model": ("model", list(mask.keys()))
    }
    return xarray.Dataset(data_vars=data_dict, coords=coord_dict)



gw_ds = maskToWBD(DATA_OUTPUT_DIR + metric_name)
gw_ds

KeyboardInterrupt: 

In [102]:
shapefile_path = SHPFILE_DIR + "WBD_USGS_HUC10_CA.shp"
geopandas.read_file(shapefile_path)["OBJECTID"]

0          1
1          2
2          3
3          4
4          5
        ... 
1123    1124
1124    1125
1125    1126
1126    1127
1127    1128
Name: OBJECTID, Length: 1128, dtype: int64

In [ ]:
xs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

for x in xs:
    